In [1]:
def read_csv(folder, filename):
    import pandas as pd

    filepath = f'{folder}/{filename}.csv'
    df = pd.read_csv(filepath)

    return df
def convert_state_code(df):

    state_param = read_csv('epidemic/linelist','param_geo')
    state_param = state_param[159:][['state','idxs']].set_index('idxs')
    state_param.state = state_param.state.str.upper()
    state_param.index = state_param.index.astype('str')

    df2 = df.assign(state=df.state.str.split(","))[['state','cluster']].explode('state')
    df2.state = df2.state.map(state_param['state']).fillna('not stated')
    df2 = df2.groupby('cluster').agg({'state': lambda x: ",".join(x)})
    df2.rename(columns={'state':'state_join'},inplace=True)

    df.set_index('cluster',inplace=True)
    df3 = df.join(df2,how = 'inner')
    df3 = df3.reset_index()[['cluster','state_join','district','date_announced','date_last_onset','category','status','cases_new','cases_total','cases_active','tests','icu','deaths','recovered','summary_bm','summary_en']]
    df3.rename(columns={'state_join':'state'},inplace=True)

    return df3

In [2]:

#statistical analysis of data
#State with the highest number of cases
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

covid_data= pd.read_csv('https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/1caf9a2289ce63da4e71339dda656be9360fa9e3/epidemic/cases_state.csv')
my_data = covid_data[covid_data['state']=='MY'].drop(['state','cases_cluster'], axis=1)
my_data = my_data[my_data.sum(axis = 1) > 0]
my_data = my_data.groupby(['state'])['cases_active'].sum().reset_index()
my_data_death = my_data[my_data['cases_active'] > 0]
state_fig = sns.displot(my_data_death, x='state', y='cases_active', title='State wise cases reported of COVID-19 in MY', text='cases_active')
state_fig.show()






KeyError: 'state'